# Training main:

take from cyanite notebook the interface to observe tags distribution and create datasets


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from torch.autograd import Variable

from frontend import Frontend_mine, Frontend_won
from backend import Backend
from data_loader import get_DataLoader

main_dict = {"frontend_dict":
             {"list_out_channels":[128,128,128,256,256,256], 
              "list_kernel_sizes":[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],
              "list_pool_sizes":  [(3,2),(2,2),(2,2),(2,2),(2,2),(2,1)], 
              "list_avgpool_flags":[False,False,False,False,False,True]},
             "backend_dict":
             {"n_class":50,
              "bert_config":None, 
              "recurrent_units":2}} # pass None to deactivate

In [2]:
class CRNNSA(nn.Module):
    
    """
    TODO: explore whether "rec_unit -> self_att -> rec_unit"
    would have worked better.
    """
    
    # BERT-based Convolutional Recurrent Neural Network
    # Code adopted from https://github.com/minzwon/sota-music-tagging-models/
    def __init__(self, main_dict=None, backend=None, frontend=None):
        super(CRNNSA, self).__init__()
        

        if main_dict is not None:
            self.frontend = Frontend_mine(main_dict["frontend_dict"])
            self.backend = Backend(main_dict)
        else:
            self.frontend = frontend
            self.backend = backend


    def forward(self, spec):
        
        x = self.backend(self.frontend(spec))
        
        return x

In [3]:
crnnsa = CRNNSA(main_dict)
print(crnnsa)

CRNNSA(
  (frontend): Frontend_mine(
    (freq_bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv_block1): BlockChoi(
      (conv): Conv2dSame(1, 128, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ELU(alpha=1.0)
      (pool): MaxPool2d(kernel_size=(3, 2), stride=(3, 2), padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (conv_block2): BlockChoi(
      (conv): Conv2dSame(128, 128, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ELU(alpha=1.0)
      (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (conv_block3): BlockChoi(
      (conv): Conv2dSame(128, 128, kernel_size=(3, 3), stride=(1, 1))
      (bn):

In [4]:
spec = torch.FloatTensor(np.load("/import/c4dm-datasets/rmri_self_att/msd/1/1/113801.npy")[np.newaxis,np.newaxis,:,:])

In [5]:
crnnsa(spec).shape

torch.Size([1, 50])

In [6]:
dataLoader = get_DataLoader(batch_size=1, mode="valid")

In [7]:
for x,y in dataLoader:
    print(x.shape)
    print(y.shape)
    print(crnnsa.frontend(x).shape)
    print(crnnsa(x).shape)
    break

torch.Size([1, 1, 96, 937])
torch.Size([1, 50])
torch.Size([1, 256, 29])
torch.Size([1, 50])



    TO SOLVE:

    /homes/lm004/.local/lib/python3.7/site-packages/torch/nn/modules/rnn.py:740: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:775.)
      self.dropout, self.training, self.bidirectional, self.batch_first)
    Traceback (most recent call last):
      File "main.py", line 252, in <module>
        solver.train()
      File "main.py", line 184, in train
        roc_auc, pr_auc = self.get_validation_auc()
      File "main.py", line 247, in get_validation_auc
        roc_auc, pr_auc = get_auc(predictions, groundtruths)
      File "main.py", line 49, in get_auc
        predictions = np.array(predictions)
    ValueError: only one element tensors can be converted to Python scalars


In [8]:
def get_auc(predictions, groundtruths):
    predictions = np.array(predictions)
    groundtruths = np.array(groundtruths)

    roc_aucs  = metrics.roc_auc_score(groundtruths, predictions, average='macro')
    pr_aucs = metrics.average_precision_score(groundtruths, predictions, average='macro')
    print('roc_auc: %.4f' % roc_aucs)
    print('pr_auc: %.4f' % pr_aucs)
    return roc_aucs, pr_aucs

def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [58]:

y_score = []
y_true = []
ctr = 0
for x,y in dataLoader:
    ctr+=1
    # forward
    #x = to_var(x)
    out = crnnsa(x)
    #out = out.detach().cpu()

    y_score.append(out.detach().numpy())

    y_true.append(y.detach().numpy())
    
    if ctr==1000:
        break 




In [59]:
y_score = np.array(y_score).squeeze()
y_true = np.array(y_true).squeeze().astype(int)

roc_aucs  = metrics.roc_auc_score(y_true,y_score, average='macro')
pr_aucs = metrics.average_precision_score(y_true, y_score, average='macro')

In [60]:
roc_aucs

0.5039593613301728

In [61]:
pr_aucs

0.04162681330544072